In [0]:

35. Configuring Spark Project Application Logs


sample logging file is found at C:\Users\Student\Desktop\Personal\DataBricks\spark-3.2.4\conf

from the file 
C:\Users\Student\Desktop\Personal\DataBricks\spark-3.2.4\conf\log4j.properties.template

i created 
log4j.properties at C:\Users\Student\Desktop\Personal\Spark\pythonProject ie in the project folder.


a proper application must have some kind of application logging. So, In this video, we are going to configure the Spark application log. Let's start. 
Spark project is using Log4J, and we also want to use the same for our Spark applications. You might be wondering, why don't we use the standard Python Logging? Why do we need to use the Log4J? I will answer this question in a minute. 
But before that, let's try to understand how we can use Log4J with PySpark? Configuring Log4J for your project is a three-step process.
Include a Log4J configuration file in your project. 
Configure your Spark to pickup the Log4J configuration file. 
And Finally, create a Python Class to get Spark's Log4J instance and use it in PySpark. 
So, let me open my Spark project and add a Log4J properties file. 

sample logging file is found at C:\Users\Student\Desktop\Personal\DataBricks\spark-3.2.4\conf
i will modify it now

I already have a preconfigured log4j.properties file. Let me add it to the project root directory. Let's look at the file content. The content of this file is pretty much standard. 
Let me quickly explain a few things. 
The Log4J works almost the same way as Python logs. 
In the most basic setup, the Log4J has got three components. 
1. Logger 
2. Configurations 
3. Appenders 
The Logger is the set of APIs which we are going to use from our application. So, I will come back to loggers and show you how to use them. Configurations are defined in the log4j properties file, and they are loaded by the loggers at runtime. Appenders are the output destinations, such as Console and a log file. These appenders are also configured in the log4j properties file. 
Log4j configurations are defined in the hierarchy. And the topmost hierarchy is the root category. So, I have defined the root category at the top. 
For any hierarchy or category, we define two things. The first thing is the log level, and the second thing is the list of appenders. 
Log4J supports multiple log levels, such as DEBUG, INFO, WARN, and ERROR. I am setting the root category as WARN. 
# Set everything to be logged to the console
So, at the topmost level, we only want to see the warnings and errors. We do not want to see the info and debug messages. And we want these messages to go to the Console. So, I am setting the appender as Console.
# define console appender 
All these configurations are standard, and they remain the same in most of the projects. So, these two sections together will set the root level log4J configuration, and they will stop all the log messages sent by the Spark and other packages except warning and errors. So, you will get a clean and minimal log output. 
# application log
However, we want to change the behavior of our application. So, I am defining the second log level specific to my application. I am naming this level as guru.learningjournal.spark.examples. This is the name that I am going to use when using the Logger in my application. I will show you that part when we start creating an application code. So, the application level log will be set to INFO. And I want to send these logs to console and file appenders. We wanted to send the root level logs to the Console only. But we want application-level logs to go to the console and log file both. We already defined the console appender. So, I am setting the file appender here. 
#Recommendations from spark template
The rest of the settings below this level are some recommendations by the Spark Log4J configuration template. So, I am keeping them in my configuration file as well. That's all we wanted to do in the log4j configurations. However, I want to talk about these variables. I am using two variables here. The first variable is to identify the log file directory location. And the second variable is to identify the log file name. All these log4j configurations are standard, and it should work well for most of your projects. 

We have only two places to tweak. The first one is the application logger name.
guru.learningjournal.spark.examples
The second one is the log file location and the log file name. 
${spark.yarn.app.container.log.dir}/${logfile.name}.log
You might be wondering, why do we need these variables? Can't' we simply give a fixed location and a file name? Let me explain. We already learned that the Spark follows a master-slave architecture, and your application is going to run as one driver and multiple executor processes. Right? And all these executors are individual JVMs running on different machines in your cluster. What does it mean? Your driver is running on one machine, and all your executors are running at different computers. But you do not have any control over it. In fact, you do not know what is going to execute where? All of that is managed by the cluster manager. Now think about the driver and executers all creating application log files on their local machines. So your log file is also distributed. How are you going to collect it? Well, you can not do it. So, you are going to rely on your cluster manager for managing the log. Now lets assume we are using YARN. So, YARN is going to collect all these log files and keep it at one predefined location so you can read them and investigate them quickly. But for YARN to find these log files, you should make sure that your application JVM is creating logs at a well defined fixed location on each machine. Right? And thats what we define here. If you want your YARN manager to collect your application logs, then you must ensure to create log files in 'spark.yarn.app.container.log.dirctory.' 
This is a variable that YARN is going to use for log collection. And this configuration should be defined appropriately by your cluster admin. If your application is creating log files at some other location, YARN is not going to collect it. Or, if your cluster admin forgot to define a value for this variable, YARN is not going to manage the logs. And you are going to struggle for receiving and reading your log files. The next variable is the log file name. We are using a variable name for the log file name because we want to control the log file name and do not wish log4j to use a default and random name. Great! We will come back to these variables once again in a later video when we are submitting our first Spark application to a real distributed cluster. But for now, we are going to run and test our application in a local machine. So, these variables should have a valid value on your local setup as well. Right?

How to configure a local Spark log directory? Spark has a complex mechanism to read configuration settings. However, at a high level, every Spark application would look for a SPARK_HOME environment variable. If you have a SPARK_HOME configured, then Spark will look into the conf directory of your SPARK_HOME. Let me show you. Here is my SPARK_HOME.
C:\Users\Student\Desktop\Personal\DataBricks\spark-3.2.4\conf
So, Spark goes to this directory, then look into the conf directory. And this is the place where Spark will look for configurations.
There is a spark-defaults.conf.template
created a duplicate of this file and renamed as spark-defaults.conf file


We will be using spark-defaults.conf file to configure the value of our log4j variables. Spark is a JVM based application. Right? It is written in Scala, and it runs in a Java Virtual Machine. So, I am going to use Spark .driver.extraJavaOptions configuration to pass on some variables to the Spark JVM.
-Dlog4j.configuration=file:log4j.properties  
The first variable tells the log4j configuration file location and name. So, What am I saying? Log4j configurations are defined in the log4j.properties file in the current directory. And that is why we added the log4j properties file in the project root directory. 
-Dspark.yarn.app.container.log.dir=app-logs 
The second variable sets the log file directory location. 
-Dlogfile.name=hello-spark
And the third one is the log file name. We are setting these variables in the JVM, but all these three variables will reach to the log4j Logger. That's how the log4j is designed. That's all. Save this file and you are done. Great! We learned a lot of things here. Let me quickly summarize. We want to use log4j for logging messages from our application. We avoid python logger because collecting python log files is not integrated with the Spark. Spark is designed to work with the log4j, and most of the cluster managers are also comfortable managing log4j files. You can still use python logger to send log messages to the Console. 
However, if you want to collect your python logs to a central location, then you must configure remote log Handlers and use them in your PySpark programs. Setting and using Python remote log handlers could be an unnecessary complexity. Using log4j in PySpark is straightforward and just a three-step process. The first step is to create a log4j configuration file and add it to your project root directory. The second step is to configure your Spark JVM to pickup your log4j properties file and other variables. The third step is to get a log4j instance and start using it in your application. We already learned the first two steps. And I will cover the third step in a separate video.